<a href="https://colab.research.google.com/github/Varunn1310/AMD/blob/main/ModelPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 1

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.233
[1,  4000] loss: 2.051
[1,  6000] loss: 1.923
[1,  8000] loss: 1.833
[1, 10000] loss: 1.774
[1, 12000] loss: 1.715
[1, 14000] loss: 1.717
[1, 16000] loss: 1.708
[1, 18000] loss: 1.634
[1, 20000] loss: 1.656
[1, 22000] loss: 1.567
[1, 24000] loss: 1.625
[1, 26000] loss: 1.633
[1, 28000] loss: 1.625
[1, 30000] loss: 1.562
[1, 32000] loss: 1.629
[1, 34000] loss: 1.614
[1, 36000] loss: 1.611
[1, 38000] loss: 1.580


KeyboardInterrupt: ignored

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
import time
def accuracy(testloader):
  correct = 0
  total = 0
  steps = 0
  throughput=[]
  start=time.time()
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = net(images)

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          steps +=1
  end=time.time()
  throughput = steps*batch_size/(end-start)
  print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
  print('Throughput is {:.4f}'.format(throughput))
  print('Latency is {:.4f}s'.format(end-start))
  acc = (100 * correct)//total
  return acc

In [ ]:
net.eval()
acc = accuracy(testloader)
print("\nAccuracy = %0.4f" % acc)

Accuracy of the network on the 10000 test images: 56 %
throughput is 838.9203
Latency is 11.9201s

Accuracy = 56.0000
